# Intra-lingual A2A VC with S3PRL; S3PRL-VC
[![Generic badge](https://img.shields.io/badge/GitHub-s3plr-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/s3prl
[notebook]:https://colab.research.google.com/github/tarepan/s3prl/blob/master/s3prl/downstream/a2a-vc-vctk/training.ipynb
[tarepan]:https://github.com/tarepan

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!python --version
!pip show torch | sed '2!d'
!/usr/local/cuda/bin/nvcc --version | sed '4!d'

## Setup

Install the package from `tarepan/s3plr` private repository

In [ ]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==1.10.0" -q      # Based on your PyTorch environment
# !pip install "torchaudio==0.10.0" -q # Based on your PyTorch environment

# repository install
!git clone https://github.com/tarepan/s3prl.git
%cd s3prl

## Preprocessing

In [ ]:
!apt-get install sox
!pip install -e ./
!pip install fairseq

%cd ./s3prl/downstream/a2a-vc-vctk

!pip install -r requirements.txt

%cd data
# Clone VCC2020 repository and extract contents
!./vcc2020_download.sh vcc2020/
# Expand VCTK-Corpus.tar.gz on `/s3prl/downstream/a2a-vc-vctk/data/`, so there is dataset directory `/s3prl/downstream/a2a-vc-vctk/data/VCTK-Corpus`
# !./vctk_download.sh ./
!cp /content/gdrive/MyDrive/ML_data/corpuses/VCTK/ver0_92/archive/VCTK-Corpus.tar.gz "."
!tar xzf ./VCTK-Corpus.tar.gz
!rm ./VCTK-Corpus.tar.gz
%cd ../

# Acquire pre-trained HiFi-GAN checkpoint file in gdrive, then expand on /s3prl/downstream/a2a-vc-vctk/hifigan_vctk/
!./vocoder_download.sh ./

In [ ]:
# ./vcc2020_download.sh 

# db=/s3prl/downstream/a2a-vc-vctk/data/vcc2020/

# srcspks=(
#     "SEF1" "SEF2" "SEM1" "SEM2"
# )

# # intra-lingual
# trgspks_task1=(
#     "TEF1" "TEF2" "TEM1" "TEM2"
# )

# # cross-lingual
# trgspks_task2=(
#     "TFF1" "TFM1" "TGF1" "TGM1" "TMF1" "TMM1"
# )

# # clone `https://github.com/nii-yamagishilab/VCC2020-database.git` on `/s3prl/downstream/a2a-vc-vctk/data/vcc2020/`
# # all zip all zip file

# /s3prl/downstream/a2a-vc-vctk/data/vcc2020/VCC2020-database

# # mv vcc2020_database_evaluation/${srcspk}/*.wav source/${srcspk}/
# # mv vcc2020_database_groundtruth/${trgspk_task1}/*.wav target_task1/${trgspk}/
# # mv vcc2020_database_groundtruth/${trgspk_task2}/*.wav target_task2/${trgspk}/

# # /
# #   source/
# #     ${srcspk}/
# #       *.wav
# #   target_task1/
# #     ${trgspk_task1}/
# #       *.wav
# #   target_task2/
# #     ${trgspk_task2}/
# #       *.wav

# # download dataset
# cwd=`pwd`

#     # move transcriptions
#     mkdir prompts
#     cat vcc2020_database_transcriptions/transcriptions_training/vcc2020_database_training_Eng_transcriptions.txt \
#         vcc2020_database_transcriptions/transcriptions_evaluation/vcc2020_database_evaluation_transcriptions.txt > prompts/Eng_transcriptions.txt
#     mv vcc2020_database_transcriptions/transcriptions_training/vcc2020_database_training_Fin_transcriptions.txt prompts/Fin_transcriptions.txt
#     mv vcc2020_database_transcriptions/transcriptions_training/vcc2020_database_training_Ger_transcriptions.txt prompts/Ger_transcriptions.txt
#     mv vcc2020_database_transcriptions/transcriptions_training/vcc2020_database_training_Man_transcriptions.txt prompts/Man_transcriptions.txt
#     rm -rf vcc2020_database_transcriptions

## Training

In [ ]:
# @./s3prl
%cd ../..

# Copy Preprocessed embeddings from storage
!cp /content/gdrive/MyDrive/ML_data/embedding_VCTK.tar.gz "."
!tar -zxvf embedding_VCTK.tar.gz
!cp /content/gdrive/MyDrive/ML_data/stats_VCTK.tar.gz "."
!tar -zxvf stats_VCTK.tar.gz

# Upstream is pretrained by s3prl
# !./downstream/a2a-vc-vctk/vc_train.sh vq_wav2vec downstream/a2a-vc-vctk/config_ar_taco2.yaml default

# stdout (Not log file) 
!mkdir -p result/downstream/a2a_vc_vctk_default_vq_wav2vec
!python run_downstream.py -m train --config downstream/a2a-vc-vctk/config_ar_taco2.yaml -p result/downstream/a2a_vc_vctk_default_vq_wav2vec -u vq_wav2vec -d a2a-vc-vctk

In [ ]:
# # Copy checkpoint
# !cp result/downstream/a2a_vc_vctk_default_vq_wav2vec/states-15000.ckpt /content/gdrive/MyDrive/ML_results/S3PRL_VC
# !ls /content/gdrive/MyDrive/ML_results/S3PRL_VC

In [ ]:
# # Launch TensorBoard
# %load_ext tensorboard
# %tensorboard --logdir gdrive/MyDrive/ML_results/S3PRL_VC

In [ ]:
# # Usage stat
# ## GPU
# !nvidia-smi -l 3
# ## CPU
# !vmstat 5
# !top